<a href="https://colab.research.google.com/github/plaban1981/INEURON/blob/master/Ineuron_session_saturday_11_01_2020_pandas_handling_nan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [3]:
df =pd.DataFrame({'A':[1,2,np.nan],'B':[5,np.nan,np.nan],'C':[1,2,3]})
df.head()

,A,B,C
0,1.0,5.0,1
1,2.0,NaN,2
2,NaN,NaN,3


In [4]:
df["Country"] = "India Australia USA".split()
df.head()

,A,B,C,Country
0,1.0,5.0,1,India
1,2.0,NaN,2,Australia
2,NaN,NaN,3,USA


In [0]:
df.set_index("Country",inplace=True)

In [6]:
df.head()

,A,B,C
Country,,,
India,1.0,5.0,1
Australia,2.0,NaN,2
USA,NaN,NaN,3


##handle nan values

In [7]:
df.dropna(axis=0)# drops a nan value found in any row

,A,B,C
Country,,,
India,1.0,5.0,1


In [8]:
df.dropna(axis = 1) #drop columns having nan values

,C
Country,
India,1
Australia,2
USA,3


In [9]:
df.dropna(axis=0,thresh=2)#mininum number of nan need to be present in a row in order to drop those values

,A,B,C
Country,,,
India,1.0,5.0,1
Australia,2.0,NaN,2


##Fill missing Values

In [10]:
df.fillna("Missing Values")

,A,B,C
Country,,,
India,1,5,1
Australia,2,Missing Values,2
USA,Missing Values,Missing Values,3


In [12]:
df.isnull() ==True

,A,B,C
Country,,,
India,False,False,False
Australia,False,True,False
USA,True,True,False


## Count the number of null values for each row

In [13]:
df.isnull().sum()

A    1
B    2
C    0
dtype: int64

In [15]:
df.isna().sum()

A    1
B    2
C    0
dtype: int64

In [17]:
np.isnan(df).sum()

A    1
B    2
C    0
dtype: int64

## inplace = True ==> makes the changes permanent

In [18]:
data = {'Company':['GOOG','GOOG','MSFT','MSFT','FB','FB'],
        'Person':['Sam','Charlie','Amy','Vanessa','Carl','Sarah'],
        'Sales': [200,120,340,124,243,350]}
df = pd.DataFrame(data)
df.describe()#only integer values

,Sales
count,6.000000
mean,229.500000
std,100.899455
min,120.000000
25%,143.000000
50%,221.500000
75%,315.750000
max,350.000000


In [0]:
group_cmp = df.groupby('Company').sum()

In [25]:
group_cmp

,Sales
Company,
FB,593
GOOG,320
MSFT,464


In [0]:
grp = df.groupby('Company').describe().loc[['MSFT','GOOG']]

In [36]:
type(grp)

pandas.core.frame.DataFrame

In [44]:
grp

Sales                                                      
        count   mean         std    min    25%    50%    75%    max
Company                                                            
MSFT      2.0  232.0  152.735065  124.0  178.0  232.0  286.0  340.0
GOOG      2.0  160.0   56.568542  120.0  140.0  160.0  180.0  200.0

In [39]:
grp.columns

MultiIndex([('Sales', 'count'),
            ('Sales',  'mean'),
            ('Sales',   'std'),
            ('Sales',   'min'),
            ('Sales',   '25%'),
            ('Sales',   '50%'),
            ('Sales',   '75%'),
            ('Sales',   'max')],
           )

In [43]:
grp.iloc[:1,2:]

Sales                                   
                std    min    25%    50%    75%    max
Company                                               
MSFT     152.735065  124.0  178.0  232.0  286.0  340.0

##pd.concat([df1,df2,df3],axis=1) --> column wise concatenation , by default axis = 0, row wise

## df1 = df.copy() - separate memory location allocated
## df1 = df ,same memory location is referenced

In [47]:
midx = pd.MultiIndex.from_tuples([(10,'Ten'),(10,'Twenty'),(20,'Ten'),(20,'Twenty')],names=['Num','Char'])
midx.to_frame(index=True)

Num    Char
Num Char               
10  Ten      10     Ten
    Twenty   10  Twenty
20  Ten      20     Ten
    Twenty   20  Twenty

In [50]:
# Creating the Tuple 
tuples =[('Physics', 85), ('Chemistry', 88), 
           ('Maths', 95), ('Computers', 99)] 
  
# Print the Tuple 
print(tuples) 
# Creating the MultiIndex 
midx = pd.MultiIndex.from_tuples(tuples, names =('Subject', 'Marks')) 
  
# Print the MultiIndex 
print(midx) 

midx.to_frame(index=True)

[('Physics', 85), ('Chemistry', 88), ('Maths', 95), ('Computers', 99)]
MultiIndex([(  'Physics', 85),
            ('Chemistry', 88),
            (    'Maths', 95),
            ('Computers', 99)],
           names=['Subject', 'Marks'])


,,Subject,Marks
Subject,Marks,,
Physics,85,Physics,85
Chemistry,88,Chemistry,88
Maths,95,Maths,95
Computers,99,Computers,99


## .apply function is applicable to Series

In [55]:
grp.index

Index(['MSFT', 'GOOG'], dtype='object', name='Company')

##How to reduce memory usage of data using Pandas?

#### 1. Read CSV file data in chunk size
To be honest, I was baffled when I encountered an error and I couldn’t read the data from CSV file, only to realize that the memory of my local machine was too small for the data with 16GB of RAM.

#### pandas.read_csv has a parameter called chunksize!

The parameter essentially means the number of rows to be read into a dataframe at any single time in order to fit into the local memory. Since the data consists of more than 70 millions of rows, I specified the chunksize as 1 million rows each time that broke the large data set into many smaller pieces.

In [0]:

# read the large csv file with specified chunksize 
df_chunk = pd.read_csv(r'../input/data.csv', chunksize=1000000)

The operation above resulted in a TextFileReader object for iteration. Strictly speaking, df_chunk is not a dataframe but an object for further operation in the next step.

By iterating each chunk, I performed data filtering/preprocessing using a function — chunk_preprocessing before appending each chunk to a list. And finally I concatenated the list into a final dataframe to fit into the local memory.

In [0]:
chunk_list = []  # append each chunk df here 

# Each chunk is in df format
for chunk in df_chunk:  
    # perform data filtering 
    chunk_filter = chunk_preprocessing(chunk)
    
    # Once the data filtering is done, append the chunk to list
    chunk_list.append(chunk_filter)
    
# concat the list into dataframe 
df_concat = pd.concat(chunk_list)

2. Filter out unimportant columns to save memory
Great. At this stage, I already had a dataframe to do all sorts of analysis required.
To save more time for data manipulation and computation, I further filtered out some unimportant columns to save more memory.


In [0]:
df = df[['col_1','col_2', 'col_3', 'col_4', 'col_5', 'col_6','col_7', 'col_8', 'col_9', 'col_10']]


3. Change dtypes for columns
The simplest way to convert a pandas column of data to a different type is to use astype().
I can say that changing data types in Pandas is extremely helpful to save memory, especially if you have large data for intense analysis or computation (For example, feed data into your machine learning model for training).